In [1]:
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession
# spark = SparkSession.builder.config("spark.driver.memory", "16g").appName('chapter_

In [2]:
prev = spark.read.csv("data/linkage/donation/block_1/block_1.csv")
prev

NameError: name 'spark' is not defined

In [ ]:
prev.show(2)

In [ ]:
parsed = spark.read.option("header", "true").option("nullValue", "?").\
option("inferSchema", "true").csv("data/linkage/donation/block_1/block_1.csv")

In [ ]:
parsed.printSchema()
parsed.show(5)

In [ ]:
parsed.count()

In [ ]:
parsed.cache()

In [ ]:
from pyspark.sql.functions import col
parsed.groupBy("is_match").count().orderBy(col("count").desc()).show()

In [ ]:
parsed.createOrReplaceTempView("linkage")
spark.sql("""
SELECT is_match, COUNT(*) cnt
FROM linkage
GROUP BY is_match
ORDER BY cnt DESC
""").show()

In [ ]:
summary = parsed.describe()

In [ ]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

In [ ]:
matches = parsed.where("is_match = true")
match_summary = matches.describe()
misses = parsed.filter(col("is_match") == False)
miss_summary = misses.describe()

In [ ]:
summary_p = summary.toPandas()

In [ ]:
summary_p.head()
summary_p.shape

In [ ]:
summary_p = summary_p.set_index('summary').transpose().reset_index()
...
summary_p = summary_p.rename(columns={'index':'field'})
...
summary_p = summary_p.rename_axis(None, axis=1)
...
summary_p.shape

In [ ]:
summaryT = spark.createDataFrame(summary_p)
summaryT
summaryT.printSchema()

In [ ]:
from pyspark.sql.types import DoubleType
for c in summaryT.columns:
 if c == 'field':
 continue
 summaryT = summaryT.withColumn(c, summaryT[c].cast(DoubleType()))
...
summaryT.printSchema()

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import DoubleType
def pivot_summary(desc):
# convert to pandas dataframe
 desc_p = desc.toPandas()
# transpose
 desc_p = desc_p.set_index('summary').transpose().reset_index()
 desc_p = desc_p.rename(columns={'index':'field'})
 desc_p = desc_p.rename_axis(None, axis=1)
# convert to Spark dataframe
 descT = spark.createDataFrame(desc_p)
# convert metric columns to double from string
 for c in descT.columns:
 if c == 'field':
 continue
 else:
 descT = descT.withColumn(c, descT[c].cast(DoubleType()))
 return desc

In [ ]:
match_summaryT = pivot_summary(match_summary)
miss_summaryT = pivot_summary(miss_summary)

In [ ]:
match_summaryT.createOrReplaceTempView("match_desc")
miss_summaryT.createOrReplaceTempView("miss_desc")
spark.sql("""
SELECT a.field, a.count + b.count total, a.mean - b.mean delta
FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
WHERE a.field NOT IN ("id_1", "id_2")
ORDER BY delta DESC, total DESC
""")

In [ ]:
good_features = ["cmp_lname_c1", "cmp_plz", "cmp_by", "cmp_bd", "cmp_bm"]
...
sum_expression = " + ".join(good_features)
...
sum_expression

In [ ]:
from pyspark.sql.functions import expr
scored = parsed.fillna(0, subset=good_features).\
withColumn('score', expr(sum_expression)).\
select('score', 'is_match')
...
scored.show()

In [ ]:
def crossTabs(scored: DataFrame, t: DoubleType) -> DataFrame:
 return scored.selectExpr(f"score >= {t} as above", "is_match").\
 groupBy("above").pivot("is_match", ("true", "false")).\
 count()

In [ ]:
crossTabs(scored, 4.0).show()
crossTabs(scored, 2.0).show()